In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
# %matplotlib inline
# import matplotlib.pyplot as plt

In [2]:
JSON_DATA_PATH = "Video_Games_5.json"
N = 10

In [3]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]        
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        rows.append((uid, iid, review, rating, dt))
        
    return pd.DataFrame(rows, columns=cols)

In [4]:
df = get_data_frame()
df.head()

,uid,iid,review,rating,dt
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400
2,2,0,1st shipment received a book instead of the ga...,1.0,1403913600
3,3,0,"I got this version instead of the PS3 version,...",3.0,1315958400
4,4,0,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,1308009600


In [5]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [6]:
training_df, test_df = split_df_by_dt(df)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 5) (46353, 5)
Размер после очистки: (185427, 5) (19174, 5)


In [7]:
#убираем игры с 1 отзывом
training_df = training_df.groupby("iid").filter(lambda x: len(x) > 1) 

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [9]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

In [10]:

def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

test_dict = get_test_dict(test_df)

In [11]:

from sklearn.preprocessing import normalize
from scipy.sparse import vstack

def _prepare_iid_data(df, review_ftr_m): 
    iid_to_row = {}
    rows = []
    # не самый оптимальный group by       
    for row_id, iid in enumerate(df.iid.unique()):
        iid_to_row[iid] = row_id
        iid_ftr_m = csr_matrix(
            review_ftr_m[np.where(df.iid == iid)[0]].sum(axis=0)
        )
        rows.append(iid_ftr_m)
    iid_ftr_m = normalize(vstack(rows, format='csr'))
    return iid_to_row, iid_ftr_m

In [12]:
def _prepare_uid_data(df, iid_to_row, iid_ftr_m):  
        uid_to_row = {}
        rows = []
        
        # gr_df - кусок df с данными одного пользователя 
        for gr_id, gr_df in df.groupby("uid"):
            uid = gr_df.uid.values[0]
            
            # поиск объектов и пользовательских рейтингов для них
            iid_rows = []
            ratings = []
            for iid, rating in zip(gr_df.iid.values, gr_df.rating.values):
                if iid in iid_to_row:
                    iid_rows.append(iid_to_row[iid])
                    ratings.append(rating)
                  
            # создание профиля пользователя
            if iid_rows:
                ratings = np.array(ratings).reshape(-1, 1)
                uid_ftr_m = csr_matrix(
                    iid_ftr_m[iid_rows].multiply(ratings).sum(axis=0)
                ) 
                uid_to_row[uid] = len(uid_to_row)
                rows.append(uid_ftr_m)
            
        uid_ftr_m = normalize(vstack(rows, format='csr'))
        return uid_to_row, uid_ftr_m

In [13]:
def get_recs(uid, top):
    recs = {}
    if uid in uid_to_row:
        u_row_id = uid_to_row[uid]
        u_row = uid_ftr_m[u_row_id]
        reviewed = list(training_df.loc[training_df['uid'] == uid]['iid'])
        fltr = len(reviewed) + top
        # самописный cosine similarity
        u_recs = u_row.dot(ftr_iid_m)
        #не добавляем в рекомендации то, что пользователь уже оценил
        for arg_id in np.argsort(-u_recs.data)[:fltr]:
            row_id = u_recs.indices[arg_id]
            if (row_to_iid[row_id] not in reviewed) & (len(recs)<10):
                score = u_recs.data[arg_id]
                recs[row_to_iid[row_id]] = score          
    return recs

In [14]:
def get_batch_recs(uids, top):
        """Строит рекомендации для нескольких пользователей uids
        :return: словарь типа {uid: {iid: score, ...}, ...}
        """
        return {uid: get_recs(uid, top) for uid in uids}

In [15]:


review_ftr_m = TfidfVectorizer(stop_words='english').fit_transform(training_df.review)

iid_to_row, iid_ftr_m = _prepare_iid_data(training_df, review_ftr_m)

uid_to_row, uid_ftr_m = _prepare_uid_data(training_df, iid_to_row, iid_ftr_m)

# немного упрощаем себе жизнь (для подсчета рекомендаций с использованием косинусной меры)
ftr_iid_m = iid_ftr_m.T.tocsr()
row_to_iid = {row_id: iid for iid, row_id in iid_to_row.items()}

recs = get_batch_recs(uids = test_df.uid.unique(), top = N)
hit_ratio(recs_dict=recs, test_dict=test_dict)

0.07703595011005136